In [ ]:
import json


In [ ]:
# load everything from json, at least 2GB of ram is needed for this
game = {}
# all keys inside game params
keys = {}
with open('tools/GameParams.json', 'r') as j:
    game = json.load(j)
    keys = game.keys()
    # just to make sure something is printed and everything are read
    print(len(keys))

ships = {}
with open('tools/ship_all.json', 'r') as j:
    ships = json.load(j)
    print(len(ships))


In [ ]:
# write json object to disk
def writeToJson(json_dict, name):
    with open(name, 'w+') as l:
        l.write(json.dumps(sort(json_dict)) + '\n')

# find keys in object, can be improved to go deep
def findKey(name, keys):
    temp = []
    for key in keys:
        if name.lower() in key.lower():
            temp.append(key)
    return temp

# find the first object from the dict
def findFirstObject(name, dic, rootMode=False):
    if isinstance(dic, dict):
        for key in dic.keys():
            temp = dic[key]
            # check if this is the object we want
            if name == key:
                if rootMode is True:
                    return dic
                else:
                    return temp
            result = findFirstObject(name, temp, rootMode)
            if result != None:
                return result
    elif isinstance(dic, list):
        # go through the list
        for value in dic:
            result = findFirstObject(name, value, rootMode)
            if result != None:
                return result

# sort dictionary by keys
def sort(d):
    if isinstance(d, dict):
        return {k : d[k] for k in sorted(d)}
    return d

In [ ]:
# upgrade
def writeUniqueUpgrades():
    # Find unique update and their ship
    unique_upgrades = findKey('Special_Mod', keys)
    # this adds ship id to legendary upgrade
    legendary = {}
    # this add legendary upgrades to ship
    ship_upgrade = {}
    # get their id and the ship they belong to
    for upgrade in unique_upgrades:
        temp = game[upgrade]
        upgrade_id = temp['id']
        upgrade_id_str = str(upgrade_id)
        # remember to add slot later
        slot = temp['slot']
        
        legendary[upgrade_id_str] = []
        for ship in temp['ships']:
            ship_id = game[ship]['id']
            ship_id_str = str(ship_id)
            
            # init
            if not ship_id_str in ship:
                ship_upgrade[ship_id_str] = {"legendary_upgrades": []}

            # you need to the slot as well
            ship_upgrade[ship_id_str]["legendary_upgrades"].append({"upgrade_id": upgrade_id_str, "slot": slot})
            legendary[upgrade_id_str].append({"ship_id": ship_id, "slot": slot})

    writeToJson(legendary, 'data/legendary_upgrade.json')
    writeToJson(ship_upgrade, 'data/ship_legendary_upgrade.json')


In [ ]:
# find all keys from data
findKey('AP', keys)

In [ ]:
# save anything to json
writeToJson(game['PAPA072_406MM_AP_MISSOURI'], 'ap_missouri.json')

In [ ]:
# preview anything
game['PJPT040_TYPE93_HAYATE']

In [ ]:
def retrieveDataFromShip(ship):
    temp = {}
    # extract permo flages
    permoflages = ship['permoflages'].copy()
    nativePermoflage = ship['nativePermoflage']
    permoflages.append(nativePermoflage)
    # we need to get all permoflage_id
    permo_ids = []
    for permo in permoflages:
        if permo != '':
            permo_ids.append(game[permo]['id'])
    # add it to temp, there is at least one
    if len(permo_ids) > 0:
        temp['permoflages'] = permo_ids

    # extract HE penetration
    artillery = findFirstObject('HitLocationArtillery', ship, rootMode=True)
    if artillery != None:
        ammoList = artillery['ammoList']
        he = [x for x in ammoList if '_HE' in x]
        # some ships doesn't have HE
        if len(he) > 0:
            # add it to temp
            temp['alphaPiercingHE'] = game[he[0]]['alphaPiercingHE']
        # get ap info here as well
        ap = [x for x in ammoList if '_AP' in x]
        if len(ap) > 0:
            saved_info = {}
            shell = game[ap[0]]
            saved_info['weight'] = shell['bulletMass']
            saved_info['diameter'] = shell['bulletDiametr']
            saved_info['drag'] = shell['bulletAirDrag']
            saved_info['velocity'] = shell['bulletSpeed']
            saved_info['krupp'] = shell['bulletKrupp']
            temp['ap'] = saved_info

    # get sigma
    sigma = findFirstObject('ammoPool', ship, rootMode=True)
    if sigma != None:
        temp['sigma'] = sigma['sigmaCount']

    # save consumables
    ability = findFirstObject('ShipAbilities', ship)
    if ability != None:
        temp['consumables'] = []
        # append slot and abilities in
        for slot in ability:
            s = ability[slot]
            # x[0] is the name of the consumable
            abils = [x for x in s['abils'] if x[0].endswith('Premium')]
            o = []
            for a in abils:
                o.append({'name': a[0], 'type': a[1]})
            # o is used to make a list of objects
            if len(o) > 0:
                # check there is at least one inside, not all ships have 5 consumables
                temp['consumables'].append(o)

    return temp

In [ ]:
# additional data
def writeShipAdditional():
    # loop through all ship 
    additional_ship_info = {}
    for ship in ships:
        ship_str = ships[ship]['ship_id_str']
        # it should only has one
        additional_ship_info[ship] = retrieveDataFromShip(game[findKey(ship_str, keys)[0]])
    writeToJson(additional_ship_info, 'data/ship_additional.json')

writeShipAdditional()

In [ ]:
# remove empty strings and any strings with .xml
def cleanupConsumables(dic):
    if isinstance(dic, dict):
        for k in dic.copy().keys():
            t = dic[k]
            if k == 'typeinfo' or k == 'group':
                del dic[k]

            if isinstance(t, str):
                if t == "" or ".xml" in t:
                    del dic[k]
            elif isinstance(t, float):
                if t == 0.0:
                    del dic[k]
            elif isinstance(t, int):
                if t == -1:
                    del dic[k]
            elif isinstance(t, list):
                if len(t) == 0:
                    del dic[k]
            else:
                cleanupConsumables(t)
    elif isinstance(dic, list):            
        for e in dic:
            cleanupConsumables(e)

In [ ]:
# save all consumables into one file and removed entries that are not needed
def writeConsumables():
    consumables = {}
    for ship in ships:
        ship_str = ships[ship]['ship_id_str']
        the_one = game[findKey(ship_str, keys)[0]]
        # find abilities
        ability = findFirstObject('ShipAbilities', the_one)
        if ability != None:
            for slot in ability:
                s = ability[slot]
                # x[0] is the name of the consumable
                abils = [x for x in s['abils'] if x[0].endswith('Premium')]
                for a in abils:
                    if not a[0] in consumables:
                        consumables[a[0]] = game[a[0]]
    cleanupConsumables(consumables)
    writeToJson(consumables, 'data/consumables.json')
    # print out all types for icons
    for c in consumables:
        t = consumables[c]
        one = t[next(iter(t.keys()))]
        print(one['consumableType'])

In [ ]:
# long range torpedoes
def writeLongTorpedos():
    long_torpedos = []
    for k in keys:
        o = game[k]
        if 'planeAmmoType' in o and o['planeAmmoType'] == 'torpedo':
            if o['maxDist'] > 399:
                dist = o['maxDist'] / 33.35
                speed = o['speed'] * 1.05
                spotted = o['visibilityFactor']
                reaction = spotted * 1000 / 2.6 / speed
                long_torpedos.append({'name': o['name'], 'dist': dist, 'speed': speed, 'spotted': spotted, 'reaction': reaction, 'type': o['typeinfo']['nation']})
    writeToJson(long_torpedos, 'extra/long_torpedoes.json')


In [ ]:
# save all commanders and legendary commanders
def writeCommanders():
    commanders = []
    legendary_commanders = []
    for key in keys:
        if 'CrewPersonality' in game[key]:
            crew = game[key]['CrewPersonality']
            # get unique and collab
            if crew['isUnique'] == True or crew['peculiarity'] != 'default':
                commanders.append(key)
            if len(game[key]['UniqueSkills'].keys()) > 0:
                legendary_commanders.append(key)
    writeToJson(commanders, 'data/commanders.json')
    writeToJson(legendary_commanders, 'data/legendary_commanders.json')

In [ ]:
# all in one
def writeAll():
    writeConsumables()
    writeShipAdditional()
    writeCommanders()
    writeLongTorpedos()
    writeUniqueUpgrades()

writeAll()